In [114]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Data Analysis

In [115]:
with open('../data/flutter/flutter_issues_labeled_2.json') as json_data:
    data = json.load(json_data)

In [116]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if issue['training_labels'][assignee] == 1:
            num_positive_labels += 1
        total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

num positive events: 819
num total events: 2990


Now filter out any issues that never end up getting a commit/PR before they are closed. Idea being that these are not 'real' issues.

In [117]:
num_positive_labels = 0
total_num_labels = 0
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                num_positive_labels += 1
            total_num_labels += 1
print("num positive events: " + str(num_positive_labels))
print("num total events: " + str(total_num_labels))

num positive events: 819
num total events: 937


In [118]:
person_counts = {}
person_totals = {}
for issue in data:
    for assignee in issue['training_labels']:
        if len(issue['matching_prs']) != 0 or len(issue['matching_commits']) != 0:
            if issue['training_labels'][assignee] == 1:
                person_counts[assignee] = person_counts.get(assignee, 0) + 1
            person_totals[assignee] = person_totals.get(assignee, 0) + 1
person_percents = {}
for person in person_totals:
    person_percents[person] = person_counts.get(person, 0) / person_totals[person]
    
print("Number of successful tasks: ")
print(person_counts)
print("Number of total tasks: ")
print(person_totals)
print("Percentage of tasks completed: ")
print(person_percents)

Number of successful tasks: 
{'Hixie': 96, 'abarth': 422, 'collinjackson': 7, 'HansMuller': 66, 'devoncarew': 1, 'mpcomplete': 20, 'yjbanov': 14, 'krisgiesing': 4, 'danrubel': 8, 'xster': 6, 'sethladd': 4, 'jason-simmons': 18, 'dragostis': 5, 'jimbeveridge': 1, 'tvolkert': 19, 'chinmaygarde': 2, 'pq': 3, 'aghassemi': 3, 'jakobr-google': 11, 'cbracken': 33, 'gspencergoog': 20, 'dvdwasibi': 1, 'johnmccutchan': 4, 'goderbauer': 13, 'lequem': 1, 'mehmetf': 3, 'mravn-google': 4, 'B3rn475': 2, 'jcollins-g': 11, 'amirh': 3, 'szakarias': 4, 'jonahwilliams': 1, 'aam': 3, 'DanTup': 2, 'mraleph': 3, 'blasten': 1}
Number of total tasks: 
{'abarth': 444, 'eseidelGoogle': 2, 'Hixie': 121, 'collinjackson': 11, 'HansMuller': 75, 'devoncarew': 2, 'vlidholt': 2, 'mpcomplete': 22, 'yjbanov': 20, 'krisgiesing': 5, 'danrubel': 9, 'xster': 17, 'sethladd': 7, 'jason-simmons': 19, 'aghassemi': 7, 'dragostis': 7, 'jimbeveridge': 1, 'tvolkert': 19, 'chinmaygarde': 7, 'pq': 3, 'jakobr-google': 11, 'johnmccutchan

In [119]:
zero_count = 0
total_count = 0
for person in person_percents:
    if person_percents[person] == 0.0:
        zero_count += 1
    total_count += 1
print(float(zero_count) / total_count)

0.18181818181818182


In [120]:
#TODO: compare characteristics of issues that are successfully completed.

In [121]:
# Think of Features - past history for dev, etc.

# Classifier

In [122]:
df_list = []
for issue in data:
    for assignee in issue['assignees']:
        df_dict = {}
        df_dict['comments'] = issue['comments']
        df_dict['assignee'] = assignee['login']
        if (not issue['body']):
            issue['body'] = ""
        df_dict['body_length'] = len(issue['body'])
        df_list.append(df_dict)
df = pd.DataFrame(df_list)
display(df.head(5))

,assignee,body_length,comments
0,abarth,27,3
1,abarth,438,2
2,jason-simmons,0,5
3,Hixie,1834,3
4,Hixie,186,0


In [123]:
#TODOs:
# One Hot Encode UserName, etc...
# Sliding windows for previous events for user.
# deal with String fields (e.g. assignee)
# Use doc classifier as a feature
# Use other features
# Compare performance to naive model (just take previous % of completion for user...)
# Move to 3 class classifier (two binary combined: can predict if they will get a PR up, will that PR be accepted)